## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from pprint import pprint
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 100_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 50_000
NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [20_000] * NUMBER_OF_PARTICIPANTS
PREFIXES = ['cosmos', 'osmo', 'terra']

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 100000
Number of threads: 100


  0%|          | 0/100000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,20000,john16505145390,QmcTMPUubiQPK2ZsEpVRDE7LdbGRBZYNRBwJSfRdRdNhYv,0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca,0xea1e4958631e349eeec2e49d59281f69eb0b453120b3...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50,osmo12asenmdatefr74qhpvr3twur35xm8t638jspu6,pottery hen display later journey drastic need...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50...,0xab18d0c72c34142df74678d8728b557e6efa5b1f01ef...,{'pub_key': 'AuE0sjrPIEMRXUysFNN9z43/gpxlkgs0e...,eyJwdWJfa2V5IjoiQXVFMHNqclBJRU1SWFV5c0ZOTjl6ND...
1,20000,john16505145391,QmcbX6Af2hx9Co1wDKnkpG5HwTVoC4sYUiJNdKiSgRgNsg,0x54a31fa6e38731c1b8eda7579cf156a33769179f,0xce2da6faaae94781d2881cc33edaf8ce32fd05f52ad1...,bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg,osmo1h48dqhht7cfukuxusujqgeuzpt43jtg8gl7kpa,arrange sure benefit recipe replace width fan ...,bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg...,0x7af0aed2bbf9b33e3e73a34cff62a19d2078f84030fa...,{'pub_key': 'AtzxTgc/gj7M7q1a6oVBOz/V9uqAWmjjP...,eyJwdWJfa2V5IjoiQXR6eFRnYy9najdNN3ExYTZvVkJPei...
2,20000,john16505145392,QmNg2da2P9CQppC5QdRsu8zy8LCbKSgwSCX7K51zexf2xq,0x7bd9105d6084694757c819f8a965e39ecf7ff8a2,0x6213c521d7e6533db5bd4af75ef0ad61251afb632a0f...,bostrom1c0h2ysdynjq7sslef0ktx86a9yk0dkp9evzwq2,cosmos1c0h2ysdynjq7sslef0ktx86a9yk0dkp96lka7d,joke vacuum town dress below either height tig...,bostrom1c0h2ysdynjq7sslef0ktx86a9yk0dkp9evzwq2...,0x73d09d6c82e35d6bd9379519a1957504ef235744e74d...,{'pub_key': 'AhE823RJVhq2gyouGMwgcDvOQK80l/hap...,eyJwdWJfa2V5IjoiQWhFODIzUkpWaHEyZ3lvdUdNd2djRH...
3,20000,john16505145393,QmbEz8ZGkWeQnpEJPSQrjnDpcyCxwyaerkmd1gxWYMn2Kf,0x60920d73ebeaede54d58d56f0a7e22b6c051a912,0x289c072a670df38e707d7c26f9c1b7ebe255431b56d6...,bostrom1na6xl94tl7r5sjxyuhva04sy8p68jhhz5a2jc3,cosmos1na6xl94tl7r5sjxyuhva04sy8p68jhhzhw7pxk,collect hockey simple firm joke air polar salo...,bostrom1na6xl94tl7r5sjxyuhva04sy8p68jhhz5a2jc3...,0x7d92f8e2e6b9db95853d5a7692bcd4c87c3b32966c58...,{'pub_key': 'A2COW5KWNuVXMJPyoNNNpsIxlcs7N7CfX...,eyJwdWJfa2V5IjoiQTJDT1c1S1dOdVZYTUpQeW9OTk5wc0...
4,20000,john16505145394,QmaBhHHPYivF8a6otweD8ZgtVWSVbB5AXpqtmVPQvfx3kB,0xa38253e4257fda031a2cf619e8dd9f455211e22a,0x3688c537e031cd40687ddcce934fba76a370e2f2d04f...,bostrom1zt4phgz5xpfzvymhnnemqesdyyw2spe8jefj4c,terra1zt4phgz5xpfzvymhnnemqesdyyw2spe8hw8pfl,recall pitch fork dune rebuild priority habit ...,bostrom1zt4phgz5xpfzvymhnnemqesdyyw2spe8jefj4c...,0xb83b211090ffee9f2e27d37552a8e346ee4604171d61...,{'pub_key': 'AynVqVXox56sBogu4VL45Uk1KD9KvvtAd...,eyJwdWJfa2V5IjoiQXluVnFWWG94NTZzQm9ndTRWTDQ1VW...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item[0],
                     'amount': str(_item[1])} for _item in claims_df[['ethereum_address', 'amount']].values.tolist()]+ \
                    [{'address': _item[0],
                      'amount': str(_item[1])} for _item in claims_df[['cosmos_address', 'amount']].values.tolist()]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 15
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(tasks)
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

{('root_testing_source_100000_addresses.json', 'temp/proofs_166.json', 166000, 167001), ('root_testing_source_100000_addresses.json', 'temp/proofs_132.json', 132000, 133001), ('root_testing_source_100000_addresses.json', 'temp/proofs_54.json', 54000, 55001), ('root_testing_source_100000_addresses.json', 'temp/proofs_120.json', 120000, 121001), ('root_testing_source_100000_addresses.json', 'temp/proofs_29.json', 29000, 30001), ('root_testing_source_100000_addresses.json', 'temp/proofs_149.json', 149000, 150001), ('root_testing_source_100000_addresses.json', 'temp/proofs_63.json', 63000, 64001), ('root_testing_source_100000_addresses.json', 'temp/proofs_145.json', 145000, 146001), ('root_testing_source_100000_addresses.json', 'temp/proofs_105.json', 105000, 106001), ('root_testing_source_100000_addresses.json', 'temp/proofs_156.json', 156000, 157001), ('root_testing_source_100000_addresses.json', 'temp/proofs_49.json', 49000, 50001), ('root_testing_source_100000_addresses.json', 'temp/pr

  0%|          | 0/200 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_72.json --start_index 72000 --end_index 73001
Merkle root: 896386ecff01a3e567efc15113b10320a6916d98b4d2bb9238e6719e352af541
Number of addresses in the Merkle tree: 1001
Done in 50.78s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_104.json --start_index 104000 --end_index 105001
Merkle root: 896386ecff01a3e567efc15113b10320a6916d98b4d2bb9238e6719e352af541
Number of addresses in the Merkle tree: 1001
Done in 50.86s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_65.json --start_index 65000 --end_index 66001
Merkle root: 896386ecff01a3e567efc15113b10320a6916d98b4d2bb9238e6719e352af541
Number of addresses in the Merkle tree: 1001
Done in 50.90s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_166.jso

In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='left',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='left',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,20000,john16505145390,QmcTMPUubiQPK2ZsEpVRDE7LdbGRBZYNRBwJSfRdRdNhYv,0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca,0xea1e4958631e349eeec2e49d59281f69eb0b453120b3...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50,osmo12asenmdatefr74qhpvr3twur35xm8t638jspu6,pottery hen display later journey drastic need...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50...,0xab18d0c72c34142df74678d8728b557e6efa5b1f01ef...,{'pub_key': 'AuE0sjrPIEMRXUysFNN9z43/gpxlkgs0e...,eyJwdWJfa2V5IjoiQXVFMHNqclBJRU1SWFV5c0ZOTjl6ND...,[d7994ad73633a407682e08454c4f6f204c62abb39ba94...,[ac0d7205b7d2da4496e9c0a99ec9309106835dfcec896...
1,20000,john16505145390,QmcTMPUubiQPK2ZsEpVRDE7LdbGRBZYNRBwJSfRdRdNhYv,0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca,0xea1e4958631e349eeec2e49d59281f69eb0b453120b3...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50,osmo12asenmdatefr74qhpvr3twur35xm8t638jspu6,pottery hen display later journey drastic need...,bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50...,0xab18d0c72c34142df74678d8728b557e6efa5b1f01ef...,{'pub_key': 'AuE0sjrPIEMRXUysFNN9z43/gpxlkgs0e...,eyJwdWJfa2V5IjoiQXVFMHNqclBJRU1SWFV5c0ZOTjl6ND...,[d7994ad73633a407682e08454c4f6f204c62abb39ba94...,[ac0d7205b7d2da4496e9c0a99ec9309106835dfcec896...
2,20000,john16505145391,QmcbX6Af2hx9Co1wDKnkpG5HwTVoC4sYUiJNdKiSgRgNsg,0x54a31fa6e38731c1b8eda7579cf156a33769179f,0xce2da6faaae94781d2881cc33edaf8ce32fd05f52ad1...,bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg,osmo1h48dqhht7cfukuxusujqgeuzpt43jtg8gl7kpa,arrange sure benefit recipe replace width fan ...,bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg...,0x7af0aed2bbf9b33e3e73a34cff62a19d2078f84030fa...,{'pub_key': 'AtzxTgc/gj7M7q1a6oVBOz/V9uqAWmjjP...,eyJwdWJfa2V5IjoiQXR6eFRnYy9najdNN3ExYTZvVkJPei...,[81f6ba90f31cfd28c01b6c2fd1f2b7969482df24c44fe...,[a12947f476bc2c7836a2e99efdc602fdec5861f394441...
3,20000,john16505145392,QmNg2da2P9CQppC5QdRsu8zy8LCbKSgwSCX7K51zexf2xq,0x7bd9105d6084694757c819f8a965e39ecf7ff8a2,0x6213c521d7e6533db5bd4af75ef0ad61251afb632a0f...,bostrom1c0h2ysdynjq7sslef0ktx86a9yk0dkp9evzwq2,cosmos1c0h2ysdynjq7sslef0ktx86a9yk0dkp96lka7d,joke vacuum town dress below either height tig...,bostrom1c0h2ysdynjq7sslef0ktx86a9yk0dkp9evzwq2...,0x73d09d6c82e35d6bd9379519a1957504ef235744e74d...,{'pub_key': 'AhE823RJVhq2gyouGMwgcDvOQK80l/hap...,eyJwdWJfa2V5IjoiQWhFODIzUkpWaHEyZ3lvdUdNd2djRH...,[cf4410fa48d4b8524be71f0b897838c9d4091b3677cef...,[6c8a772467c91ba7d401ae87f4bfa6480946d96b03dbc...
4,20000,john16505145393,QmbEz8ZGkWeQnpEJPSQrjnDpcyCxwyaerkmd1gxWYMn2Kf,0x60920d73ebeaede54d58d56f0a7e22b6c051a912,0x289c072a670df38e707d7c26f9c1b7ebe255431b56d6...,bostrom1na6xl94tl7r5sjxyuhva04sy8p68jhhz5a2jc3,cosmos1na6xl94tl7r5sjxyuhva04sy8p68jhhzhw7pxk,collect hockey simple firm joke air polar salo...,bostrom1na6xl94tl7r5sjxyuhva04sy8p68jhhz5a2jc3...,0x7d92f8e2e6b9db95853d5a7692bcd4c87c3b32966c58...,{'pub_key': 'A2COW5KWNuVXMJPyoNNNpsIxlcs7N7CfX...,eyJwdWJfa2V5IjoiQTJDT1c1S1dOdVZYTUpQeW9OTk5wc0...,[0c9925d293ac5afdd7c80530610f28408c84b3b138530...,[cf6588c49f84523a5264a7b4fd92dbd811c7c0b3ba23f...


In [6]:
for i in range(ceil(NUMBER_OF_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [10]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1923ktlaaw5gsnsxtl5evudr798s9vp744q4rtu9mapt24p2heq3sccue2w


### Initiate Class of Output Parsing

In [11]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [12]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 896386ecff01a3e567efc15113b10320a6916d98b4d2bb9238e6719e352af541
Gas used: 121,492
Tx hash: 04115509D4B1BAF1CDECB5334585561BADF092583A42AE2BE81B0BD16D411C17


#### Get Merkle Root form the Gift Contract

In [13]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1923ktlaaw5gsnsxtl5evudr798s9vp744q4rtu9mapt24p2heq3sccue2w
{'data': {'merkle_root': '896386ecff01a3e567efc15113b10320a6916d98b4d2bb9238e6719e352af541'}}


### Send coins to new addresses

In [19]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1000

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=bostrom_addresses_item,
            amounts=[1] * len(bostrom_addresses_item),
            gas=min(100_000 + 25_000 * len(bostrom_addresses_item), int(20e6)),
            display_data=DISPLAY_TX_EXECUTION)


  0%|          | 0/101 [00:00<?, ?it/s]

In [20]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom175vtwuws8cmwmf77z4gwl9tcanvffrqcrjfy7s
	amount: 1boot
	receiver: bostrom1wtg70symffpautvx29yhe97lvr4n49mcrduqw4
	amount: 1boot
	receiver: bostrom1dwp5yzr8uzet6d6ke6w5qquqn3qt8tqk6jc2zq
	amount: 1boot
	receiver: bostrom1du97aryhcyjm2nllq2m9uynkppc0jvnfeyz2dg
	amount: 1boot
	receiver: bostrom18mgg8g39ysmjyh98g3rkwjkdt3fxxxvmndavc8
	amount: 1boot
	receiver: bostrom1p92uzx7nnwhq0vh6gaszfhxl0k3drqf3kljlx7
	amount: 1boot
	receiver: bostrom1y9wcs0fwupa2a45uth0nueyfulg2udwmcj9gle
	amount: 1boot
	receiver: bostrom1rryc7e89tu5z5mwn672rnj5arvzl3a4j4290j4
	amount: 1boot
	receiver: bostrom12hk72aqafsy63kgy0hl9g57t34u2cyc6hj0kdf
	amount: 1boot
	receiver: bostrom16r0uqwha3z4xlj628e85e7fs0zx4asljzvgj4v
	amount: 1boot
	receiver: bostrom15e0n3guazqjtdygqspfhy8wcejz3nh87g43xwe
	amount: 1boot
	receiver: bostrom1pdv9revfdwuy723gy6ex5r8mwtsq9nttve4wht
	amount: 1boot
	receiver: bostrom124dk6j0wvapv8gll57nhk827dwn3tk0jklzqna
	amount: 1boot
	receiver: bostrom16r3h4taj3

### Create Passports, Proof Addresses and Claim Gift

In [21]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 50000
Number of threads: 100


  0%|          | 0/50000 [00:00<?, ?it/s]

In [22]:
with open('temp/contract_participation_execution_log_1.txt', 'r') as f:
    data = json.load(f)

In [23]:
pprint(data)

{'claim_cosmos': '{"code":32,"codespace":"sdk","data":null,"gas_used":66365,"gas_wanted":400000,"height":0,"info":null,"logs":null,"raw_log":"account '
                 'sequence mismatch, expected 3, got 2: incorrect account '
                 'sequence","timestamp":null,"txhash":"A567FA0EE76102306A2FE2FE5504C6E1DBEF718B099F753492ACC64025044B99"}',
 'claim_ethereum': '{"code":32,"codespace":"sdk","data":null,"gas_used":66345,"gas_wanted":400000,"height":0,"info":null,"logs":null,"raw_log":"account '
                   'sequence mismatch, expected 3, got 2: incorrect account '
                   'sequence","timestamp":null,"txhash":"CED8ECE2EFD05EDA28712583242FA4B5B4437B2870D314E000DC11396CB29B74"}',
 'create': '{"code":0,"codespace":"","data":null,"gas_used":440639,"gas_wanted":500000,"height":2315386,"info":null,"logs":[{"events":[{"attributes":[{"key":"particleFrom","value":"Qmarkb2T79Wqpb1qPuwmtadXNvEGmHoMjVNVVzFGwbv8Y3"},{"key":"particleTo","value":"QmT5kx9Pcp2wcHPontSqDsNhwXqsT4r

In [30]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 273502
Gas used: 440,639
Tx hash: C5D37ED01DA7729B4EFF34D23F914D91ECA5F05276919F6C246A9C18773602DE


In [38]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

cyberlinks
	Nickname -> Ethereum Address
	Ethereum Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16505145390
	address: 0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca
Gas used: 281,286
Tx hash: FE3E533881DB9739E4BA17DC8CB90F74EE0FA5E36E303EE7116A6ED0F8FB3754


In [31]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events
account sequence mismatch, expected 3, got 2: incorrect account sequence
Gas used: 45,725
Tx hash: 194F45CEAF88E1D67E6800105DF9F19F9A2C60F902CD76CAA0618059A7AF47C8


In [32]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events
account sequence mismatch, expected 3, got 2: incorrect account sequence
Gas used: 66,365
Tx hash: A567FA0EE76102306A2FE2FE5504C6E1DBEF718B099F753492ACC64025044B99


In [35]:
res_participation[1]

"{'claim': {'nickname': 'john16505145390', 'gift_claiming_address': '0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca', 'gift_amount': '20000', 'proof': ['d7994ad73633a407682e08454c4f6f204c62abb39ba9445ffbd752682e7399f6', 'b216b4d989f96eea1236bfd7f3353511062b6ccf57144b27e5fc4696bc628db3', '304d2eb8c435271af3c3a5a26637b93c5180485c65b6e6d0d8c0c2e5543d43a3', 'c1911041c63aed27e6525c4e53b779ee4a83e6bc56928815931052d130425fd6', 'dd4acb78e0ba298c05a87aab437939f2247135ee95db77cbf1a497b4b24d7223', '95fdb1deacc6b5e579f311c48ab34e05d60e8b06601930db6d815aabed949ff7', '5790cd3f7f1a87cd1c9790f7871ddd3e77c435788eb50c6046614a741c1307b4', '475f65dbbd92f3cc6993fcc9b555c5e90e0fcf1bc1f3b6f6af1cb38c66d66941', 'f90ea473b36d04ad3fad9569be7763273ba88288b7d4a1445644d8d89d3b066e', 'e46722181b39fec39867bd85c42b39f300a9d2973abec6649000b1d5ba508f51', '1c34554ac02a8a87fa7007a99808b334608d4b294ecf4cf3ce168a46f59dc297', '025ad62bb9bc634ccd872746b721f97a3179b13f4ad0cc8579d354eb750cda99', '34b67b797b94ce2d67f718e15005ede24c0

### Release Gift

In [27]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 50000
Number of threads: 100


  0%|          | 0/50000 [00:00<?, ?it/s]

In [28]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [33]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	amount: 15628boot

coin spent
	spender: Gift Contract
	amount: 15628boot

execute
	execute contract: Gift Contract

message from bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	sender: Gift Contract
	amount: 15628boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	gift_address: 0x97649bc00f28d5df4ec7918f9b7781fc8ec98aca
	stage: 9
	amount: 15,628
Gas used: 157,446
Tx hash: C0FC762311E6F9A4A7394671D3906826927A62CF1DE9280DD2A34FA74BF3947B


In [34]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	amount: 15471boot

coin spent
	spender: Gift Contract
	amount: 15471boot

execute
	execute contract: Gift Contract

message from bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	sender: Gift Contract
	amount: 15471boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	gift_address: osmo12asenmdatefr74qhpvr3twur35xm8t638jspu6
	stage: 9
	amount: 15,471
Gas used: 157,541
Tx hash: DD3319B0873F418047473499B7CE76E22466FB42128B6F7AF95A58C75453D0F1


## Passport NFT testing
### Transfer Passport

In [40]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [41]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom12asenmdatefr74qhpvr3twur35xm8t63v6hz50
	recipient: Passport Owner Address
	token_id: 273502
Gas used: 386,208
Tx hash: 397994D783B8B06CFCD80DB336466FA3CBAD3C8192A9DC8590C2853A0534117A


### Burn

In [42]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1h48dqhht7cfukuxusujqgeuzpt43jtg8rhe4fg
	token_id: 273515
Gas used: 382,043
Tx hash: 5166979A21AF9222CEA51AC262101239CCEE2CF69F46CBBA346F4DF0A0B1A297


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
